In [ ]:
# python 3

In [1]:
# 创建环境并定义维度参数
# create the env. and define its dimension parameters

In [2]:
import gym

In [3]:
env_name = "Pendulum-v0"

In [4]:
env = gym.make( env_name )

[2018-11-14 17:28:14,532] Making new env: Pendulum-v0


In [5]:
env.state_dim = env.observation_space.shape[0]
env.action_dim = env.action_space.shape[0]
env.DoF = 1

In [9]:
# 充当服务器的功能
# work as a servicer

In [6]:
import numpy as np
import socket
import threading
import json

In [7]:
TCP_PORT = 12005

In [11]:
def env_servicer(env):
    #Create The Socket
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)

    #Listen The Port
    s.bind(('',TCP_PORT))
    s.listen(5)
    print('Waiting for connection...')

    def tcplink(sock,addr):
        print('Accept new connection from %s:%s...' % addr)
        sock.send( ('Welcome!').encode() )
        while True:
            data=sock.recv(1024).decode()
            print("data: ", data)

            data_json = json.loads( data )
            print('data_json["type"]: ', data_json["type"])

            if data_json["type"] == "init":
                data = { 'state_dim' : env.state_dim, 'action_dim' : env.action_dim, 'DoF' : env.DoF } 

            elif data_json["type"] == "reset":
                state = env.reset()
                data = { 'state' : state.tolist() } 

            elif data_json["type"] == "step":
                a = np.array( data_json["action"] )
                state_next, r, done, info = env.step(a)
                data = { 'state' : state_next.tolist(), 'reward' : r, 'done' : done, 'info' : info } 
            elif data_json["type"] == "close":
                break
                
            str_json = json.dumps(data)
            sock.send( str_json.encode() )
        sock.close()
        print('Connection from %s:%s closed.'%addr)
        
    while True:
        # Accept A New Connection
        sock,addr=s.accept()
        
        # Create A New Thread to Deal with The TCP Connection
        t=threading.Thread(target=tcplink(sock,addr))

In [ ]:
env_servicer( env )

Waiting for connection...
Accept new connection from 127.0.0.1:47396...
data:  {"type": "init"}
data_json["type"]:  init
data:  {"type": "reset"}
data_json["type"]:  reset
data:  {"type": "reset"}
data_json["type"]:  reset
data:  {"action": [0.1], "type": "step"}
data_json["type"]:  step
data:  {"action": [0.1], "type": "step"}
data_json["type"]:  step
data:  {"action": [0.1], "type": "step"}
data_json["type"]:  step


In [ ]:
# json 转换示例01

In [22]:
data = '{"type": "init"}'

In [23]:
data_json = json.loads( data )

In [24]:
data_json["type"]

'init'

In [ ]:
# json 转换示例02

In [10]:
data = {'a':True, 'b':False, 'c':None, 'd':(1,2), 1:'abc'}
# help(data): class dict(object)

In [14]:
data_str = json.dumps( data )

In [17]:
data_json = json.loads( data_str )

In [18]:
data_json["a"]

True